# Entries in circulant 1d mass- and inter-/histopolation matrices

In [ ]:
from mpi4py import MPI
import numpy as np

from struphy.psydac_api.psydac_derham import Derham
from struphy.psydac_api.mass import WeightedMassOperator

from psydac.ddm.cart import DomainDecomposition
from psydac.fem.tensor import TensorFemSpace

In [ ]:
# instance of Derham
Nel = [12, 12, 12]              # Number of grid cells
p = [1, 2, 3]                   # spline degrees
spl_kind = [True, True, True]   # Spline types (clamped vs. periodic)
comm = MPI.COMM_WORLD

derham = Derham(Nel, p, spl_kind, comm=comm)

In [ ]:
# 1d fem spaces

# H1 in each direction
V0_fem = derham.Vh_fem['0'].spaces
print(V0_fem)

# L2 in each direction
V3_fem = derham.Vh_fem['3'].spaces
print(V3_fem)

In [ ]:
# 1d mass matrices in H1 (no weight)
mass_H1_1d = []
for femspace_1d in V0_fem:

    domain_decompos_1d = DomainDecomposition([femspace_1d.ncells], [femspace_1d.periodic])
    femspace_1d_tensor = TensorFemSpace(domain_decompos_1d, femspace_1d, quad_order=[femspace_1d.degree])

    M = WeightedMassOperator(femspace_1d_tensor, femspace_1d_tensor)
    M.assemble(verbose=False)
    M.matrix.exchange_assembly_data()
    
    mass_H1_1d += [M.matrix.toarray()]

# 1d mass matrices in L2 (no weight)
mass_L2_1d = []
for femspace_1d in V3_fem:

    domain_decompos_1d = DomainDecomposition([femspace_1d.ncells], [femspace_1d.periodic])
    femspace_1d_tensor = TensorFemSpace(domain_decompos_1d, femspace_1d, quad_order=[femspace_1d.degree])

    M = WeightedMassOperator(femspace_1d_tensor, femspace_1d_tensor)
    M.assemble(verbose=False)
    M.matrix.exchange_assembly_data()
    
    mass_L2_1d += [M.matrix.toarray()]

In [ ]:
print('Sorted eigenvalues of H1 mass matrices in 1d:')
for deg, M in zip(p, mass_H1_1d):
    print(f'\np={deg}:\n', np.sort(np.linalg.eigvals(M)))

print('\nSorted eigenvalues of L2 mass matrices in 1d:')
for deg, M in zip(p, mass_L2_1d):
    print(f'\np={deg}:\n', np.sort(np.linalg.eigvals(M)))

In [ ]:
print('First row of circulant H1 mass matrices in 1d:')
for deg, M in zip(p, mass_H1_1d):
    print(f'\np={deg}:\n', M[0])

print('\nFirst row of circulant L2 mass matrices in 1d:')
for deg, M in zip(p, mass_L2_1d):
    print(f'\np={deg}:\n', M[0])

In [ ]:
# 1d Inter-/hitopolation matrices

# Commuting projectors
P0 = derham.P['0']
P3 = derham.P['3']

# 1d collocation matrices
colloc_H1_1d = []
for mat in P0._imat.mats:
    colloc_H1_1d += [mat.toarray()]

# 1d histopolation matrices
histop_L2_1d = []
for mat in P3._imat.mats:
    histop_L2_1d += [mat.toarray()]

In [ ]:
print('First row of circulant H1 collocation matrices in 1d:')
for deg, M in zip(p, colloc_H1_1d):
    print(f'\np={deg}:\n', M[0])

print('\nFirst row of circulant L2 histopolation matrices in 1d:')
for deg, M in zip(p, histop_L2_1d):
    print(f'\np={deg}:\n', M[0])